conda_tensorflow_p36

In [1]:
import subprocess, json

In [2]:
def add_class(method):
    return "public class Test {\n"+method+"\n}"

In [3]:
def remove_class(parse_tree):
    return {'classbody': parse_tree['compilationUnit'][0]['typeDeclaration'][1]['classDeclaration'][2]['classBody'][1:-1]}

In [4]:
def parse(code):
    code = add_class(code)
    tree = subprocess.check_output(['java', '-jar', 'mavenproject1-1.0-SNAPSHOT-jar-with-dependencies.jar', code])
    tree = json.loads(tree)
    tree = remove_class(tree)
    return tree

In [18]:
import ast
def tree2code(tree, indentation):
    code = ""
    parent = list(tree.keys())[0]
    children = tree[parent]
    if 'type' in tree:
        code = tree['text'] + ' '
        if tree['text'] in [';', '{', '}']:
            code += '\n'
            if tree['text'] == '{':
                indentation += 1
            elif tree['text'] == '}':
                indentation -= 1
            code += '\t'*indentation
        #return code, indentation
    elif len(children)==1 and parent == "literal":
        child = children[0]
        child_text = child["text"]
        #type(ast.literal_eval(child_text))
        try:
            if child_text in ['null', 'true', 'false']:
                code = child_text + " "
            elif type(ast.literal_eval(child_text)) == type("STRING"):
                code = "STRING00 "
            elif type(ast.literal_eval(child_text)) == type(1):
                code = "INT00 "
            elif type(ast.literal_eval(child_text)) == type(1.1):
                code = "FLOAT00 "
        except:
            code = "NUM00 "
    else:
        root = list(tree.keys())[0]
        children = tree[root]
        for child in children:
            t, indentation = tree2code(child, indentation)
            code += t
    return code, indentation

In [32]:
import json
import copy
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data

def dump_dict(data, name):
    with open(name, 'w', encoding="utf8") as f:
        json.dump(data, f)

In [17]:
## Special Tokens
# STRING
# _NUM

In [13]:
code = """
private int findPLV ( int M_PriceList_ID ) { Timestamp priceDate = null ; String dateStr = Env . getContext ( Env . getCtx ( ) , p_WindowNo , STRING ) ; if ( dateStr != null && dateStr . length ( ) > _NUM ) priceDate = Env . getContextAsDate ( Env . getCtx ( ) , p_WindowNo , STRING ) ; else { dateStr = Env . getContext ( Env . getCtx ( ) , p_WindowNo , STRING ) ; if ( dateStr != null && dateStr . length ( ) > _NUM ) priceDate = Env . getContextAsDate ( Env . getCtx ( ) , p_WindowNo , STRING ) ; } if ( priceDate == null ) priceDate = new Timestamp ( System . currentTimeMillis ( ) ) ; log . config ( STRING + M_PriceList_ID + STRING + priceDate ) ; int retValue = _NUM ; String sql = STRING + STRING + STRING + STRING + STRING + STRING ; try { PreparedStatement pstmt = DB . prepareStatement ( sql , null ) ; pstmt . setInt ( _NUM , M_PriceList_ID ) ; ResultSet rs = pstmt . executeQuery ( ) ; while ( rs . next ( ) && retValue == _NUM ) { Timestamp plDate = rs . getTimestamp ( _NUM ) ; if ( ! priceDate . before ( plDate ) ) retValue = rs . getInt ( _NUM ) ; } rs . close ( ) ; pstmt . close ( ) ; } catch ( SQLException e ) { log . log ( Level . SEVERE , sql , e ) ; } Env . setContext ( Env . getCtx ( ) , p_WindowNo , STRING , retValue ) ; return retValue ; }
"""
tree = parse(code)
tree

{'classbody': [{'classBodyDeclaration': [{'modifier': [{'classOrInterfaceModifier': [{'type': 33,
         'text': 'private'}]}]},
    {'memberDeclaration': [{'methodDeclaration': [{'typeSpec': [{'primitiveType': [{'type': 27,
             'text': 'int'}]}]},
        {'type': 100, 'text': 'findPLV'},
        {'formalParameters': [{'type': 57, 'text': '('},
          {'formalParameterList': [{'formalParameter': [{'typeSpec': [{'primitiveType': [{'type': 27,
                   'text': 'int'}]}]},
              {'variableDeclaratorId': [{'type': 100,
                 'text': 'M_PriceList_ID'}]}]}]},
          {'type': 58, 'text': ')'}]},
        {'methodBody': [{'block': [{'type': 59, 'text': '{'},
            {'blockStatement': [{'localVariableDeclarationStatement': [{'localVariableDeclaration': [{'typeSpec': [{'classOrInterfaceType': [{'type': 100,
                       'text': 'Timestamp'}]}]},
                  {'variableDeclarators': [{'variableDeclarator': [{'variableDeclaratorId':

In [14]:
code = tree2code(tree,-1)[0]
print(code)

private int findPLV ( int M_PriceList_ID ) { 
Timestamp priceDate = null ; 
String dateStr = Env . getContext ( Env . getCtx ( ) , p_WindowNo , STRING ) ; 
if ( dateStr != null && dateStr . length ( ) > _NUM ) priceDate = Env . getContextAsDate ( Env . getCtx ( ) , p_WindowNo , STRING ) ; 
else { 
	dateStr = Env . getContext ( Env . getCtx ( ) , p_WindowNo , STRING ) ; 
	if ( dateStr != null && dateStr . length ( ) > _NUM ) priceDate = Env . getContextAsDate ( Env . getCtx ( ) , p_WindowNo , STRING ) ; 
	} 
if ( priceDate == null ) priceDate = new Timestamp ( System . currentTimeMillis ( ) ) ; 
log . config ( STRING + M_PriceList_ID + STRING + priceDate ) ; 
int retValue = _NUM ; 
String sql = STRING + STRING + STRING + STRING + STRING + STRING ; 
try { 
	PreparedStatement pstmt = DB . prepareStatement ( sql , null ) ; 
	pstmt . setInt ( _NUM , M_PriceList_ID ) ; 
	ResultSet rs = pstmt . executeQuery ( ) ; 
	while ( rs . next ( ) && retValue == _NUM ) { 
		Timestamp plDate = rs . getTi

In [ ]:
# add class
# parse
# build dictionary
# Build Numto compress
# compress
# tokenize
# remove class tokens

# Building Dictionary

In [33]:
# Use glob to find all files
# Or just list the 6 directories manually
paths = [
    'code_summarization/dev/code.original',
    'code_summarization/test/code.original',
    'code_summarization/train/code.original'
]
paths

['code_summarization/dev/code.original',
 'code_summarization/test/code.original',
 'code_summarization/train/code.original']

In [27]:
import datetime
start = datetime.datetime.now()

#path = 'code_summarization/train/code.original'
for path in paths:
    print("=====[", path,"]=====")
    i = 0
    error_count = 0
    parse_trees = []
    codes = []
    with open(path) as data:
        for raw_code in data:
            try:
                end = datetime.datetime.now()
                if i%500 == 0: print("[", i, "]", end-start)
                raw_code = raw_code.replace("STRING", "\"STRING\"").replace("_NUM", "1")
                tree = parse(raw_code)
                code = tree2code(tree, -1)[0]
            except Exception as e:
                print(e)
                print("Line:",i,"is corrupted. Cannot build dictionary.")
                error_count += 1
                tree = "{}"
                code = ""
            parse_trees.append(tree)
            codes.append({"code": code})
            i+=1

    dump_dict(parse_trees, path.replace("original", "tree"))
    dump_dict(codes, path.replace("original", "noliteral"))
    end = datetime.datetime.now()
    print("Dictionary building complete in - ",end-start)
    print("Error Count", error_count)

=====[ code_summarization/train/code.original ]=====
[ 0 ] 0:00:00.004962
[ 10 ] 0:00:03.144819
[ 20 ] 0:00:06.131856
[ 30 ] 0:00:09.035163
[ 40 ] 0:00:12.174366
[ 50 ] 0:00:15.118951
[ 60 ] 0:00:18.082444
[ 70 ] 0:00:21.593979
[ 80 ] 0:00:24.614260
[ 90 ] 0:00:27.634592
[ 100 ] 0:00:30.705538
[ 110 ] 0:00:33.702318
[ 120 ] 0:00:36.707486
[ 130 ] 0:00:39.649449
[ 140 ] 0:00:42.576207
[ 150 ] 0:00:45.809872
[ 160 ] 0:00:48.721481
[ 170 ] 0:00:51.962971
[ 180 ] 0:00:55.693547
[ 190 ] 0:00:59.109560
[ 200 ] 0:01:02.156657
[ 210 ] 0:01:05.227154
[ 220 ] 0:01:08.286420
[ 230 ] 0:01:11.629214
[ 240 ] 0:01:14.700897
[ 250 ] 0:01:17.477642
[ 260 ] 0:01:20.522065
[ 270 ] 0:01:23.529190
[ 280 ] 0:01:26.737890
[ 290 ] 0:01:29.676745
[ 300 ] 0:01:32.752194
[ 310 ] 0:01:35.729214
[ 320 ] 0:01:38.822734
[ 330 ] 0:01:42.023546
[ 340 ] 0:01:45.078328
[ 350 ] 0:01:48.228394
[ 360 ] 0:01:51.472957
[ 370 ] 0:01:54.534087


KeyboardInterrupt: 

In [35]:
codes[0:10]

[{'code': ('@ Override public int runCommand ( boolean mergeErrorIntoOutput , String ... commands ) throws IOException , InterruptedException { \nreturn runCommand ( mergeErrorIntoOutput , new ArrayList < String > ( Arrays . asList ( commands ) ) ) ; \n} \n',
   -1)},
 {'code': ('private int findPLV ( int M_PriceList_ID ) { \nTimestamp priceDate = null ; \nString dateStr = Env . getContext ( Env . getCtx ( ) , p_WindowNo , STRING00 ) ; \nif ( dateStr != null && dateStr . length ( ) > INT00 ) priceDate = Env . getContextAsDate ( Env . getCtx ( ) , p_WindowNo , STRING00 ) ; \nelse { \n\tdateStr = Env . getContext ( Env . getCtx ( ) , p_WindowNo , STRING00 ) ; \n\tif ( dateStr != null && dateStr . length ( ) > INT00 ) priceDate = Env . getContextAsDate ( Env . getCtx ( ) , p_WindowNo , STRING00 ) ; \n\t} \nif ( priceDate == null ) priceDate = new Timestamp ( System . currentTimeMillis ( ) ) ; \nlog . config ( STRING00 + M_PriceList_ID + STRING00 + priceDate ) ; \nint retValue = INT00 ; \n

In [ ]:
## build freq_dict


In [ ]:
## add func_names from abstract_code_dict, set count: 0

for i in range in len(abstract_code_dict):
    func = '$F'+str(i)
    freq_dict[func] = 0

In [ ]:
## initialize vocab_size
vocab_size = 50000

In [ ]:
import re

num_to_compress = 0
for i in range(len(abstract_code_keys)):
    func_name = abstract_code_dict[abstract_code_keys[i]]['compressed_name']
    code = abstract_code_keys[i]
    count = abstract_code_dict[abstract_code_keys[i]]['count']
    most_frequent_list = sorted(freq_dict, key = lambda x:freq_dict[x], reverse=True)
    
    if count >= freq_dict[most_frequent_list[vocab_size]]:
        token_list = [x for x in code.split() if x[:3] != '$id']
        for token in token_list:
            freq_dict[token] -= count
        freq_dict[func_name] += count
    else:
        num_to_compress = i
        print("Number to Compress:",i)
        break
most_frequent_list = sorted(freq_dict, key = lambda x:freq_dict[x], reverse=True)